### 네이버 검색결과 크롤링 - 수원 맛집

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup


In [2]:
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='+quote('수원맛집')
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [3]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- selenium으로 데이터 가져오기

In [4]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [5]:
# View 페이지를 아래로 내려 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [6]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

In [7]:
driver.close()

- 날짜, 제목, 컨텐츠

In [9]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [10]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [11]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜 처리

In [13]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2023, 4, 25, 12, 11, 2, 962545)

In [15]:
# 어제
date = today - timedelta(1)
f'{date.year}.{date.month}.{date.day}'

'2023.4.24'

In [19]:
# 23시간 전
date = today - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [17]:
# 2일 전
date = today - timedelta(2)
f'{date.year}.{date.month}.{date.day}'

'2023.4.23'

In [18]:
date = today - timedelta(minutes=45)
f'{date.year}.{date.month}.{date.day}'

'2023.4.25'

In [ ]:
def convert_date(s):
    if len(s) == 11:
        